In [1]:
#Importações das Bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.tree as skl
%matplotlib inline
#%matplotlib notebook

# Módulo 07, Tarefa 01

Vamos começar a mexer na nossa base de projeto? Já fizemos um exercício de montar a variável resposta, já pudemos perceber que essa atividade pode não ser trivial. Vamos agora trabalhar a base para que fique propícia ao *scikitlearn* para trabalharmos.

Lembrando, a base se chama demo01.csv, e originalmente está publicada [aqui](https://www.kaggle.com/rikdifos/credit-card-approval-prediction).

#### 1) Carregue a base e avalie:

- As variáveis
- Tipos de dados de cada variável
- Quantidade de missings
- Distribuição da variável resposta (mau)

In [2]:
#Lendo o CSV e Amostra de Dados
df = pd.read_csv('demo01.csv')
df.head()

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,possui_celular,possui_fone_comercial,possui_fone,possui_email,qt_pessoas_residencia,mau
0,M,Y,Y,0,Working,Secondary / secondary special,Married,House / apartment,58.832877,3.106849,1,0,0,0,2.0,False
1,F,N,Y,0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,52.356164,8.358904,1,0,1,1,1.0,False
2,F,N,Y,0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,52.356164,8.358904,1,0,1,1,1.0,False
3,M,Y,Y,0,Working,Higher education,Married,House / apartment,46.224658,2.106849,1,1,1,1,2.0,False
4,F,Y,N,0,Working,Incomplete higher,Married,House / apartment,29.230137,3.021918,1,0,0,0,2.0,False


In [3]:
#Analisando Quantidade de Linhas, Colunas, Tipos de Dados e NA.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16650 entries, 0 to 16649
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sexo                   16650 non-null  object 
 1   posse_de_veiculo       16650 non-null  object 
 2   posse_de_imovel        16650 non-null  object 
 3   qtd_filhos             16650 non-null  int64  
 4   tipo_renda             16650 non-null  object 
 5   educacao               16650 non-null  object 
 6   estado_civil           16650 non-null  object 
 7   tipo_residencia        16650 non-null  object 
 8   idade                  16650 non-null  float64
 9   tempo_emprego          16650 non-null  float64
 10  possui_celular         16650 non-null  int64  
 11  possui_fone_comercial  16650 non-null  int64  
 12  possui_fone            16650 non-null  int64  
 13  possui_email           16650 non-null  int64  
 14  qt_pessoas_residencia  16650 non-null  float64
 15  ma

In [4]:
#Tratando as duplicadas
print(f"Antes de retirar as duplicadas:\n{df['mau'].value_counts()}\nTotal: {df.shape[0]}")
df_tratado = df.drop_duplicates().copy()
df_tratado.reset_index(drop=True, inplace=True)
print(f"\nDepois de retirar as duplicadas:\n{df_tratado['mau'].value_counts()}\nTotal: {df_tratado.shape[0]}")

Antes de retirar as duplicadas:
False    16260
True       390
Name: mau, dtype: int64
Total: 16650

Depois de retirar as duplicadas:
False    6472
True      298
Name: mau, dtype: int64
Total: 6770


In [5]:
#Transformando [object e bool] em int.
df_tratado['sexo'] = df_tratado['sexo'].map({'F': 1, 'M':0})
df_tratado['posse_de_veiculo'] = df_tratado['posse_de_veiculo'].map({'Y': 1, 'N':0})
df_tratado['posse_de_imovel'] = df_tratado['posse_de_imovel'].map({'Y': 1, 'N':0})
df_tratado['mau'] = df_tratado['mau'].map({True: 1, False:0})
df_tratado[['sexo', 'posse_de_veiculo', 'posse_de_imovel', 'mau']]

,sexo,posse_de_veiculo,posse_de_imovel,mau
0,0,1,1,0
1,1,0,1,0
2,0,1,1,0
3,1,1,0,0
4,1,0,1,0
...,...,...,...,...
6765,1,0,1,1
6766,1,0,1,1
6767,1,0,1,1
6768,0,1,1,1


#### 2) Vamos montar um metadados

1. Crie um dataframe com os nomes de cada variável e o tipo de dados de cada variável.
2. Adicione uma coluna nesse *dataframe* chamada "qtd_categorias" e coloque nela o número de categorias correspondente de cada variável. 
    Dica: 
        1. inicie uma lista vazia
        2. faça um for ao longo dos nomes das variáveis, 
        3. conte o número de categorias dessa variável 
        4. acumule essa informação de 3. na lista que você criou em 1. 
        5. No final, essa lista pode ser atribuída à nossa variável.
3. Crie variáveis dummy para as variáveis necessárias (i.e. aquelas que são qualitativas e não estão armazenadas como {0, 1} ou {True, False}.

In [6]:
#1. e 2. Criação de DataFrame com os nomes de cada variável mostrando os Tipos de Dados e 'qtd_categorias'.
variaveis = pd.DataFrame(df_tratado.dtypes, columns=['Dtype'])
variaveis['qtd_categorias'] = None
for i in variaveis.index:
    variaveis.loc[i, 'qtd_categorias'] = df_tratado[i].nunique()
variaveis

,Dtype,qtd_categorias
sexo,int64,2
posse_de_veiculo,int64,2
posse_de_imovel,int64,2
qtd_filhos,int64,8
tipo_renda,object,5
educacao,object,5
estado_civil,object,5
tipo_residencia,object,6
idade,float64,5298
tempo_emprego,float64,3005


#### 3) Crie variáveis dummy para as variáveis necessárias (i.e. aquelas que são qualitativas e não estão armazenadas como {0, 1} ou {True, False}. Crie um *dataframe* apenas com as variáveis apropriadas para entrada no scikitlearn - elimine as variáveis tipo *str*, mantendo apenas suas versões *dummy*.

In [7]:
#3. Criação do df_dummy para as variáveis do tipo object.
df_dummy = pd.get_dummies(df_tratado, columns=['tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia'])
df_dummy.dtypes

sexo                                        int64
posse_de_veiculo                            int64
posse_de_imovel                             int64
qtd_filhos                                  int64
idade                                     float64
tempo_emprego                             float64
possui_celular                              int64
possui_fone_comercial                       int64
possui_fone                                 int64
possui_email                                int64
qt_pessoas_residencia                     float64
mau                                         int64
tipo_renda_Commercial associate             uint8
tipo_renda_Pensioner                        uint8
tipo_renda_State servant                    uint8
tipo_renda_Student                          uint8
tipo_renda_Working                          uint8
educacao_Academic degree                    uint8
educacao_Higher education                   uint8
educacao_Incomplete higher                  uint8


#### 4) Qual variável é mais poderosa?

Considere as variáveis ```possui_email``` e ```posse_de_veiculo```. Faça uma tabela cruzada entre elas e responda qual delas te parece mais poderosa para prever a probabilidade de ```mau = 1```?

In [8]:
#Comparando as duas variáveis com a variável 'mau',
#a variável mais poderosa para prever a probabilidade de mau = 1 é 'possui_email', pois tem mais inadimplentes.
freq1 = pd.crosstab(df_dummy['possui_email'], df_dummy['mau'])
print(f'{freq1}\n')
freq2 = pd.crosstab(df_dummy['posse_de_veiculo'], df_dummy['mau'])
print(f'{freq2}\n')
print(f'Mau pagar que não possui email:\n{freq1.values[0,1]}\n')
print(f'Mau pagar que não possui posse de veiculo:\n{freq2.values[0,1]}\n')

mau              0    1
possui_email           
0             5927  278
1              545   20

mau                  0    1
posse_de_veiculo           
0                 4021  190
1                 2451  108

Mau pagar que não possui email:
278

Mau pagar que não possui posse de veiculo:
190



#### 5) Salve a base, pois ela será utilizada no final deste módulo.

In [9]:
#Salvando a base de dados tratada.
df_dummy.to_csv('df_dummy.csv',index=False)

In [10]:
#Teste do CSV salvo.
pd.read_csv('df_dummy.csv')

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,idade,tempo_emprego,possui_celular,possui_fone_comercial,possui_fone,possui_email,...,estado_civil_Married,estado_civil_Separated,estado_civil_Single / not married,estado_civil_Widow,tipo_residencia_Co-op apartment,tipo_residencia_House / apartment,tipo_residencia_Municipal apartment,tipo_residencia_Office apartment,tipo_residencia_Rented apartment,tipo_residencia_With parents
0,0,1,1,0,58.832877,3.106849,1,0,0,0,...,1,0,0,0,0,1,0,0,0,0
1,1,0,1,0,52.356164,8.358904,1,0,1,1,...,0,0,1,0,0,1,0,0,0,0
2,0,1,1,0,46.224658,2.106849,1,1,1,1,...,1,0,0,0,0,1,0,0,0,0
3,1,1,0,0,29.230137,3.021918,1,0,0,0,...,1,0,0,0,0,1,0,0,0,0
4,1,0,1,0,27.482192,4.024658,1,0,1,0,...,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6765,1,0,1,1,34.857534,3.101370,1,1,1,0,...,0,0,1,0,0,1,0,0,0,0
6766,1,0,1,0,54.109589,9.884932,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
6767,1,0,1,0,43.389041,7.380822,1,1,1,0,...,1,0,0,0,0,1,0,0,0,0
6768,0,1,1,0,30.005479,9.800000,1,1,0,0,...,1,0,0,0,0,1,0,0,0,0
